# Ornstein-Uhlenbeck Process

In [7]:
import numpy as np

In [6]:
from ing.models.ou import OrnsteinUhlenbeck
from ing.fit.transition_density import ExactDensity
from ing.fit.mle_estimator import MLE

from ing.utils.data_utils import read_excel_to_series, strip_data

In [2]:
FILE_PATH = "../data/spread.xlsx"
COLUMN = "Spread"

In [3]:
df = read_excel_to_series(file_path=FILE_PATH)
df

,Unnamed: 0,Spread
0,0,20.380000
1,1,20.400000
2,2,20.412500
3,3,20.418125
4,4,20.428125
...,...,...
29177,29177,22.083125
29178,29178,22.073125
29179,29179,22.087500
29180,29180,22.133125


In [15]:
sample = strip_data(df=df, column=COLUMN)

In [25]:
param_bounds = [(0.0, 10), (0.0, 10), (0.01, 3)]
guess = np.array([0.001, 0.001, 0.002])

In [26]:
dt = 1.0 / 78
model = OrnsteinUhlenbeck()

In [28]:
exact_est = MLE(sample, param_bounds, dt, density=ExactDensity(model)).estimate_params(
    guess
)

Initial Params: [0.001 0.001 0.002]
Initial Likelihood: -1883959.8568307795
`xtol` termination condition is satisfied.
Number of iterations: 34, function evaluations: 112, CG iterations: 58, optimality: 1.11e-04, constraint violation: 0.00e+00, execution time: 0.089 s.
Final Params: [1.28685532e-06 1.69527495e-01 4.58293369e-01]
Final Likelihood: 44646.3972155862
